In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv("supplement.csv")
dataset.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52


In [3]:
dataset.describe()

,Store_id,Holiday,#Order,Sales
count,188340.000000,188340.000000,188340.000000,188340.000000
mean,183.000000,0.131783,68.205692,42784.327982
std,105.366308,0.338256,30.467415,18456.708302
min,1.000000,0.000000,0.000000,0.000000
25%,92.000000,0.000000,48.000000,30426.000000
50%,183.000000,0.000000,63.000000,39678.000000
75%,274.000000,0.000000,82.000000,51909.000000
max,365.000000,1.000000,371.000000,247215.000000


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188340 entries, 0 to 188339
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             188340 non-null  object 
 1   Store_id       188340 non-null  int64  
 2   Store_Type     188340 non-null  object 
 3   Location_Type  188340 non-null  object 
 4   Region_Code    188340 non-null  object 
 5   Date           188340 non-null  object 
 6   Holiday        188340 non-null  int64  
 7   Discount       188340 non-null  object 
 8   #Order         188340 non-null  int64  
 9   Sales          188340 non-null  float64
dtypes: float64(1), int64(3), object(6)
memory usage: 14.4+ MB


In [5]:
import plotly.express as px

In [6]:
pie = dataset["Store_id"].value_counts()
store = pie.index
orders = pie.values

fig = px.pie(dataset, values=orders, names=store)

In [7]:
dataset['Discount'] = dataset['Discount'].map({"Yes":1, "No":0})

In [8]:
dataset

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,1,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,1,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,1,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,1,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,1,62,45614.52
...,...,...,...,...,...,...,...,...,...,...
188335,T1188336,149,S2,L3,R2,2019-05-31,1,1,51,37272.00
188336,T1188337,153,S4,L2,R1,2019-05-31,1,0,90,54572.64
188337,T1188338,154,S1,L3,R2,2019-05-31,1,0,56,31624.56
188338,T1188339,155,S3,L1,R2,2019-05-31,1,1,70,49162.41


In [9]:
dataset['Store_Type'] = dataset['Store_Type'].map({"S1":1, "S2":2, "S3":3, "S4":4})

In [10]:
dataset['Location_Type'] = dataset['Location_Type'].map({"L1":1, "L2":2, "L3":3, "L4":4})

In [11]:
import sklearn

In [12]:
# Bulding the ML modal
from sklearn.model_selection import train_test_split

In [13]:
x = np.array(dataset[["Store_Type", "Location_Type", "Holiday", "Discount"]])
y = np.array(dataset["#Order"])

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [15]:
y_test

array([ 54, 111,  59, ...,  40,  69,  68], dtype=int64)

In [16]:
import lightgbm as ltb
model = ltb.LGBMRegressor()

In [17]:
model.fit(x_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001817 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 15
[LightGBM] [Info] Number of data points in the train set: 150672, number of used features: 4
[LightGBM] [Info] Start training from score 68.163401


LGBMRegressor()

In [18]:
y_pred = model.predict(x_test)

In [23]:
y_pred

array([47.35221153, 97.06885397, 66.57783665, ..., 47.35221153,
       61.74944784, 85.34103588])

In [19]:
predicted_data = pd.DataFrame(data={"predicted Orders": y_pred.flatten()})

In [20]:
predicted_data

,predicted Orders
0,47.352212
1,97.068854
2,66.577837
3,85.142955
4,54.451144
...,...
37663,66.577837
37664,47.722356
37665,47.352212
37666,61.749448


In [21]:
y_test

array([ 54, 111,  59, ...,  40,  69,  68], dtype=int64)

In [22]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)
final = pd.DataFrame([y_pred, y_test])
final

,0,1,2,3,4,5,6,7,8,9,...,37658,37659,37660,37661,37662,37663,37664,37665,37666,37667
0,47.352212,97.068854,66.577837,85.142955,54.451144,85.341036,54.451144,54.451144,85.142955,45.225135,...,47.722356,103.721016,44.418562,85.446416,50.906925,66.577837,47.722356,47.352212,61.749448,85.341036
1,54.000000,111.000000,59.000000,67.000000,60.000000,83.000000,58.000000,58.000000,50.000000,25.000000,...,39.000000,86.000000,27.000000,68.000000,29.000000,75.000000,58.000000,40.000000,69.000000,68.000000
